In [1]:
from spark_utils import get_spark_session
import pyspark.sql.functions as F

spark = get_spark_session()
# run rank_by_latest.py if this doesn't exist
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_latest')
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/11 15:27:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
assert train_data.count() == train_data.select('customer_ID').distinct().count()
assert train_labels.count() == train_labels.select('customer_ID').distinct().count()
assert train_data.count() == train_data.join(train_labels, on='customer_ID', how='inner').count()

In [3]:
%%time
from format_data import CATEGORICAL_VARIABLES
from encoder import CategoricalToIntegerEncoders

encs = CategoricalToIntegerEncoders(columns=CATEGORICAL_VARIABLES).fit(train_data)
# not actually transformed yet, just fitted
train_data_transformed = encs.transform(spark=spark, df=train_data)

CPU times: user 244 ms, sys: 48.3 ms, total: 292 ms
Wall time: 4.54 s


In [4]:
%%time
train_pdf = encs.transform(
    spark=spark,
    df=train_data.join(train_labels, on='customer_ID', how='inner')
).toPandas()

22/06/11 15:27:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 1.15 s, sys: 1.2 s, total: 2.35 s
Wall time: 1min 12s


In [5]:
from format_data import TARGET_VARIABLE, DATE_VARIABLES, ID_VARIABLES

feature_columns = [
    c for c in train_pdf.columns 
    if c not in [TARGET_VARIABLE,] + ID_VARIABLES + list(DATE_VARIABLES.keys())
]
', '.join(feature_columns)

'P_2, D_39, B_1, B_2, R_1, S_3, D_41, B_3, D_42, D_43, D_44, B_4, D_45, B_5, R_2, D_46, D_47, D_48, D_49, B_6, B_7, B_8, D_50, D_51, B_9, R_3, D_52, P_3, B_10, D_53, S_5, B_11, S_6, D_54, R_4, S_7, B_12, S_8, D_55, D_56, B_13, R_5, D_58, S_9, B_14, D_59, D_60, D_61, B_15, S_11, D_62, D_65, B_16, B_17, B_18, B_19, B_20, S_12, R_6, S_13, B_21, D_69, B_22, D_70, D_71, D_72, S_15, B_23, D_73, P_4, D_74, D_75, D_76, B_24, R_7, D_77, B_25, B_26, D_78, D_79, R_8, R_9, S_16, D_80, R_10, R_11, B_27, D_81, D_82, S_17, R_12, B_28, R_13, D_83, R_14, R_15, D_84, R_16, B_29, S_18, D_86, D_87, R_17, R_18, D_88, B_31, S_19, R_19, B_32, S_20, R_20, R_21, B_33, D_89, R_22, R_23, D_91, D_92, D_93, D_94, R_24, R_25, D_96, S_22, S_23, S_24, S_25, S_26, D_102, D_103, D_104, D_105, D_106, D_107, B_36, B_37, R_26, R_27, D_108, D_109, D_110, D_111, B_39, D_112, B_40, S_27, D_113, D_115, D_118, D_119, D_121, D_122, D_123, D_124, D_125, D_127, D_128, D_129, B_41, B_42, D_130, D_131, D_132, D_133, R_28, D_134, D_

In [6]:
X = train_pdf[feature_columns]
y = train_pdf[TARGET_VARIABLE]
print(y.unique())
# Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.
w = y.apply(lambda x: 20. if x == 1. else 1.)
print(w.unique())

[1. 0.]
[20.  1.]


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid, w_train, w_valid = train_test_split(X, y, w)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, w_train.shape, w_valid.shape

((344184, 188), (114729, 188), (344184,), (114729,), (344184,), (114729,))

In [8]:
%%time
from lightgbm import LGBMClassifier

m = LGBMClassifier().fit(
    X=X_train, y=y_train, sample_weight=w_train,
    categorical_feature = encs.columns_encoded
)
m.score(X=X_train, y=y_train, sample_weight=w_train), m.score(X=X_valid, y=y_valid, sample_weight=w_valid)

/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


CPU times: user 41.6 s, sys: 5.7 s, total: 47.3 s
Wall time: 18.3 s


(0.955786019241124, 0.9513086169002068)

In [9]:
from evaluation import amex_metric
from format_data import TARGET_VARIABLE, PREDICTION_VARIABLE
import pandas as pd


def evaluate(X, y_true) -> float:
    y_true = y_true.reset_index(drop=True)
    y_pred = m.predict_proba(X)[:, 1]

    y_true = pd.DataFrame({TARGET_VARIABLE: y_true})
    y_pred = pd.DataFrame({PREDICTION_VARIABLE: y_pred})
    return amex_metric(y_true=y_true, y_pred=y_pred)


evaluate(X_train, y_train)


0.784301470625004